### INSURANCE POLICY LAPSE PREDICTION

### Import Packages

In [147]:
#A00
#import basic packages.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss


### Import Files

In [2]:
#A00
#import files
client_df = pd.read_csv('Practice/data/client_data.csv')
payment_df = pd.read_csv('Practice/data/payment_history.csv')
policy_df = pd.read_csv('Practice/data/policy_data.csv')
main_train_df = pd.read_csv('Practice/data/train.csv')
smp_sub_df = pd.read_csv('Practice/data/sample_sub.csv')

### Expolaratory Data Analysis

In [3]:
#EDA on main_train_df

main_train_df.head(3)

,Policy ID,Lapse,Lapse Year
0,PID_4928TWH,?,?
1,PID_KBLLEGK,?,?
2,PID_90F0QA3,?,?


The Policy ID with Lapse and Lapse Year '?' values are meant to be in the test df and will be separated

In [4]:
main_train_df.shape

(51685, 3)

The train df has 51,685 rows and 3 columns

In [5]:
main_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51685 entries, 0 to 51684
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Policy ID   51685 non-null  object
 1   Lapse       51685 non-null  object
 2   Lapse Year  51685 non-null  object
dtypes: object(3)
memory usage: 1.2+ MB


In [6]:
main_train_df.isna().sum()

Policy ID     0
Lapse         0
Lapse Year    0
dtype: int64

No null values

In [7]:
len(main_train_df['Policy ID'].unique())

51685

There are no duplicate Policy ID values

In [8]:
main_train_df['Lapse'].unique()

array(['?', '1'], dtype=object)

Apart from '?', the Lapse value is equal to 1

In [9]:
main_train_df['Lapse Year'].unique()

array(['?', '2019', '2018', '2017'], dtype=object)

The Lapse Years are between (2017-2019)

### Feature Engineering

First, we create the test df. Our target variable is Lapse, therefore we drop the Lapse Year column

In [10]:
#A00
test_df = main_train_df[main_train_df['Lapse'] == '?'].drop(['Lapse Year'], axis =1)
test_df.head(3)

,Policy ID,Lapse
0,PID_4928TWH,?
1,PID_KBLLEGK,?
2,PID_90F0QA3,?


In [11]:
test_df['Lapse'].unique()

array(['?'], dtype=object)

In [12]:
test_df.shape

(43707, 2)

Out of the 51,685 rows, 43,707 are in our test df

We create the train_df without the test rows

In [70]:
#A00
train_df = main_train_df[main_train_df['Lapse'] != '?']
print(train_df.shape)
train_df.head(3)

(7978, 3)


,Policy ID,Lapse,Lapse Year
16,PID_MFAAYNJ,1,2019
23,PID_TICDPAY,1,2019
30,PID_SPACC3N,1,2018


In [71]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7978 entries, 16 to 51680
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Policy ID   7978 non-null   object
 1   Lapse       7978 non-null   object
 2   Lapse Year  7978 non-null   object
dtypes: object(3)
memory usage: 249.3+ KB


Our train df has 7,978 rows. The Lapse and Lapse Year are stored as objects. We will convert them to integers

In [72]:
#A00
#Converting Lapse & Lapse Year to integers
train_df.loc[:, ('Lapse Year')] = pd.to_numeric(train_df['Lapse Year'])
train_df.loc[:, ('Lapse')] = pd.to_numeric(train_df['Lapse'])

# Converting DATEPAI to datetime format
payment_df['DATEPAID'] = pd.to_datetime(payment_df['DATEPAID'])

train_df.info()

C:\Users\USER\SeraApps\anaconda\lib\site-packages\pandas\core\indexing.py:1843: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
C:\Users\USER\SeraApps\anaconda\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 7978 entries, 16 to 51680
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Policy ID   7978 non-null   object
 1   Lapse       7978 non-null   int64 
 2   Lapse Year  7978 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 249.3+ KB


Success!

In [73]:
train_df['Lapse'].unique()

array([1], dtype=int64)

In [74]:
train_df['Lapse Year'].unique()

array([2019, 2018, 2017], dtype=int64)

We combine the following main features to the train_df: <br>
POLICY_ID, LAPSE, LAPSE_YEAR - from train_df <br/>
SEX, BIRTH_DATE - from client_df <br/>
AMOUNT_PAID, DATE_PAID, PREMIUM_DUE_DATE - from payment_df <br/>
PRODUCT_CODE(PPR_PRODCD), PRINC_RSHIP(CLF_LIFECD), SUM_ASSURED(NPR_SUM_ASSURE), CATEGORY - from policy_df


In [75]:
#A00 
#Combine main columns in the datasets to the train_df
client_df_features = ['Policy ID','NPH_SEX','NPH_BIRTHDATE']
payment_df_features = ['Policy ID', 'AMOUNTPAID', 'DATEPAID', 'PREMIUMDUEDATE']
policy_df_features = ['Policy ID', 'PPR_PRODCD', 'CLF_LIFECD', 'NPR_SUMASSURED', 'CATEGORY']

#Join the Client Dataset
train_df = train_df.join(
    client_df[client_df_features].set_index('Policy ID'),
    on = 'Policy ID', how = 'inner')

#Join the Payment Dataset
train_df = train_df.join(
    payment_df[payment_df_features].set_index('Policy ID'),
    on = 'Policy ID', how = 'inner')

#Join the Policy Dataset
train_df = train_df.join(
    policy_df[policy_df_features].set_index('Policy ID'),
    on = 'Policy ID', how = 'inner')

#Rename the columns
rename_cols = {
    'Policy ID': 'POLICY_ID', 'NPH_SEX': 'GENDER',
    'NPH_BIRTHDATE': 'BIRTH YEAR', 'AMOUNTPAID': 'AMOUNT_PAID',
    'DATEPAID': 'DATE_PAID', 'PREMIUMDUEDATE': 'PREMIUM_DUE_DATE',
    'PPR_PRODCD': 'PRODUCT_CODE', 'CLF_LIFECD': 'PRINC_RSHIP',
    'NPR_SUMASSURED': 'SUM_ASSURED', 'Lapse': 'LAPSE', 'Lapse Year': 'LAPSE_YEAR'
}

train_df.rename(columns=rename_cols, inplace=True)

#Re-order the features
train_df = train_df.reindex(columns=
    ['POLICY_ID'] + [col for col in train_df.columns if col not in\
        ['POLICY_ID', 'LAPSE', 'LAPSE_YEAR']] + ['LAPSE_YEAR'] + ['LAPSE']
    )

train_df

 

,POLICY_ID,GENDER,BIRTH YEAR,AMOUNT_PAID,DATE_PAID,PREMIUM_DUE_DATE,PRODUCT_CODE,PRINC_RSHIP,SUM_ASSURED,CATEGORY,LAPSE_YEAR,LAPSE
16,PID_MFAAYNJ,F,1987,0.0,2018-01-10,2018-01-10 00:00:00,PPR_PRODCD_KOFUYNN,4,27631.086529,CATEGORY_GWW4FYB,2019,1
16,PID_MFAAYNJ,F,1987,0.0,2018-01-10,2018-01-10 00:00:00,PPR_PRODCD_KOFUYNN,1,123487.430359,CATEGORY_GWW4FYB,2019,1
16,PID_MFAAYNJ,F,1987,0.0,2018-01-10,2018-01-10 00:00:00,PPR_PRODCD_KOFUYNN,2,123487.430359,CATEGORY_GWW4FYB,2019,1
16,PID_MFAAYNJ,F,1987,0.0,2018-01-10,2018-01-10 00:00:00,PPR_PRODCD_B2KVCE7,2,NaN,CATEGORY_GWW4FYB,2019,1
16,PID_MFAAYNJ,F,1987,0.0,2018-01-10,2018-01-10 00:00:00,PPR_PRODCD_B2KVCE7,3,NaN,CATEGORY_GWW4FYB,2019,1
...,...,...,...,...,...,...,...,...,...,...,...,...
51671,PID_BS40NIJ,F,1984,0.0,2018-01-11,2018-01-11 00:00:00,PPR_PRODCD_KOFUYNN,2,27631.086529,CATEGORY_GWW4FYB,2019,1
51671,PID_BS40NIJ,F,1984,0.0,2018-01-11,2018-01-11 00:00:00,PPR_PRODCD_B2KVCE7,4,NaN,CATEGORY_GWW4FYB,2019,1
51671,PID_BS40NIJ,F,1984,0.0,2018-01-11,2018-01-11 00:00:00,PPR_PRODCD_KOFUYNN,3,27631.086529,CATEGORY_GWW4FYB,2019,1
51671,PID_BS40NIJ,F,1984,0.0,2018-01-11,2018-01-11 00:00:00,PPR_PRODCD_KOFUYNN,1,27631.086529,CATEGORY_GWW4FYB,2019,1


In [76]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 460479 entries, 16 to 51671
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   POLICY_ID         460479 non-null  object        
 1   GENDER            460479 non-null  object        
 2   BIRTH YEAR        460479 non-null  int64         
 3   AMOUNT_PAID       457287 non-null  float64       
 4   DATE_PAID         460479 non-null  datetime64[ns]
 5   PREMIUM_DUE_DATE  451293 non-null  object        
 6   PRODUCT_CODE      460479 non-null  object        
 7   PRINC_RSHIP       460479 non-null  int64         
 8   SUM_ASSURED       276432 non-null  float64       
 9   CATEGORY          460479 non-null  object        
 10  LAPSE_YEAR        460479 non-null  int64         
 11  LAPSE             460479 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(4), object(5)
memory usage: 45.7+ MB


##### We get a summary of the dataframe

In [78]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 460479 entries, 16 to 51671
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   POLICY_ID         460479 non-null  object        
 1   GENDER            460479 non-null  object        
 2   BIRTH YEAR        460479 non-null  int64         
 3   AMOUNT_PAID       457287 non-null  float64       
 4   DATE_PAID         460479 non-null  datetime64[ns]
 5   PREMIUM_DUE_DATE  451293 non-null  object        
 6   PRODUCT_CODE      460479 non-null  object        
 7   PRINC_RSHIP       460479 non-null  int64         
 8   SUM_ASSURED       276432 non-null  float64       
 9   CATEGORY          460479 non-null  object        
 10  LAPSE_YEAR        460479 non-null  int64         
 11  LAPSE             460479 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(4), object(5)
memory usage: 45.7+ MB


In [79]:
#A00
# unique values in each columns
new_df = train_df[['GENDER','PRODUCT_CODE','PRINC_RSHIP','CATEGORY']]
for x in new_df.columns:
    #prinfting unique values
    print(x ,':', len(new_df[x].unique()))

GENDER : 2
PRODUCT_CODE : 9
PRINC_RSHIP : 6
CATEGORY : 6


In [80]:
#A00
#One-Hot encoding the categorical parameters using get_dummies()
one_hot_encoded_data = pd.get_dummies(train_df, columns = ['GENDER','PRODUCT_CODE','CATEGORY'])


In [81]:
#reorder columns

column_titles = [col for col in one_hot_encoded_data.columns if col!= 'LAPSE'] + ['LAPSE']
train_df=one_hot_encoded_data.reindex(columns=column_titles)
train_df


,POLICY_ID,BIRTH YEAR,AMOUNT_PAID,DATE_PAID,PREMIUM_DUE_DATE,PRINC_RSHIP,SUM_ASSURED,LAPSE_YEAR,GENDER_F,GENDER_M,...,PRODUCT_CODE_PPR_PRODCD_KOFUYNN,PRODUCT_CODE_PPR_PRODCD_OK3TM96,PRODUCT_CODE_PPR_PRODCD_W0F6GK1,CATEGORY_CATEGORY_1750CEH,CATEGORY_CATEGORY_8DALFYO,CATEGORY_CATEGORY_GWW4FYB,CATEGORY_CATEGORY_LXSLG6M,CATEGORY_CATEGORY_M1ZXYVG,CATEGORY_CATEGORY_R821UZV,LAPSE
16,PID_MFAAYNJ,1987,0.0,2018-01-10,2018-01-10 00:00:00,4,27631.086529,2019,1,0,...,1,0,0,0,0,1,0,0,0,1
16,PID_MFAAYNJ,1987,0.0,2018-01-10,2018-01-10 00:00:00,1,123487.430359,2019,1,0,...,1,0,0,0,0,1,0,0,0,1
16,PID_MFAAYNJ,1987,0.0,2018-01-10,2018-01-10 00:00:00,2,123487.430359,2019,1,0,...,1,0,0,0,0,1,0,0,0,1
16,PID_MFAAYNJ,1987,0.0,2018-01-10,2018-01-10 00:00:00,2,NaN,2019,1,0,...,0,0,0,0,0,1,0,0,0,1
16,PID_MFAAYNJ,1987,0.0,2018-01-10,2018-01-10 00:00:00,3,NaN,2019,1,0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51671,PID_BS40NIJ,1984,0.0,2018-01-11,2018-01-11 00:00:00,2,27631.086529,2019,1,0,...,1,0,0,0,0,1,0,0,0,1
51671,PID_BS40NIJ,1984,0.0,2018-01-11,2018-01-11 00:00:00,4,NaN,2019,1,0,...,0,0,0,0,0,1,0,0,0,1
51671,PID_BS40NIJ,1984,0.0,2018-01-11,2018-01-11 00:00:00,3,27631.086529,2019,1,0,...,1,0,0,0,0,1,0,0,0,1
51671,PID_BS40NIJ,1984,0.0,2018-01-11,2018-01-11 00:00:00,1,27631.086529,2019,1,0,...,1,0,0,0,0,1,0,0,0,1


In [82]:
#A00
#drop the rows with amount_paid = 0 values
train_df = train_df[train_df.AMOUNT_PAID != 0]
train_df

,POLICY_ID,BIRTH YEAR,AMOUNT_PAID,DATE_PAID,PREMIUM_DUE_DATE,PRINC_RSHIP,SUM_ASSURED,LAPSE_YEAR,GENDER_F,GENDER_M,...,PRODUCT_CODE_PPR_PRODCD_KOFUYNN,PRODUCT_CODE_PPR_PRODCD_OK3TM96,PRODUCT_CODE_PPR_PRODCD_W0F6GK1,CATEGORY_CATEGORY_1750CEH,CATEGORY_CATEGORY_8DALFYO,CATEGORY_CATEGORY_GWW4FYB,CATEGORY_CATEGORY_LXSLG6M,CATEGORY_CATEGORY_M1ZXYVG,CATEGORY_CATEGORY_R821UZV,LAPSE
16,PID_MFAAYNJ,1987,3517.916006,2018-09-29,2018-01-10 00:00:00,4,27631.086529,2019,1,0,...,1,0,0,0,0,1,0,0,0,1
16,PID_MFAAYNJ,1987,3517.916006,2018-09-29,2018-01-10 00:00:00,1,123487.430359,2019,1,0,...,1,0,0,0,0,1,0,0,0,1
16,PID_MFAAYNJ,1987,3517.916006,2018-09-29,2018-01-10 00:00:00,2,123487.430359,2019,1,0,...,1,0,0,0,0,1,0,0,0,1
16,PID_MFAAYNJ,1987,3517.916006,2018-09-29,2018-01-10 00:00:00,2,NaN,2019,1,0,...,0,0,0,0,0,1,0,0,0,1
16,PID_MFAAYNJ,1987,3517.916006,2018-09-29,2018-01-10 00:00:00,3,NaN,2019,1,0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51671,PID_BS40NIJ,1984,2341.902952,2018-12-24,2019-01-01 00:00:00,2,27631.086529,2019,1,0,...,1,0,0,0,0,1,0,0,0,1
51671,PID_BS40NIJ,1984,2341.902952,2018-12-24,2019-01-01 00:00:00,4,NaN,2019,1,0,...,0,0,0,0,0,1,0,0,0,1
51671,PID_BS40NIJ,1984,2341.902952,2018-12-24,2019-01-01 00:00:00,3,27631.086529,2019,1,0,...,1,0,0,0,0,1,0,0,0,1
51671,PID_BS40NIJ,1984,2341.902952,2018-12-24,2019-01-01 00:00:00,1,27631.086529,2019,1,0,...,1,0,0,0,0,1,0,0,0,1


##### We do have missing values that need to be filled i.e. 'SUM_ASSURED ' & 'AMOUNT_PAID '

In [83]:
# train_df[train_df['AMOUNT_PAID'].isna()]
train_df.loc[train_df['POLICY_ID'] == 'PID_K5MU9OI']

,POLICY_ID,BIRTH YEAR,AMOUNT_PAID,DATE_PAID,PREMIUM_DUE_DATE,PRINC_RSHIP,SUM_ASSURED,LAPSE_YEAR,GENDER_F,GENDER_M,...,PRODUCT_CODE_PPR_PRODCD_KOFUYNN,PRODUCT_CODE_PPR_PRODCD_OK3TM96,PRODUCT_CODE_PPR_PRODCD_W0F6GK1,CATEGORY_CATEGORY_1750CEH,CATEGORY_CATEGORY_8DALFYO,CATEGORY_CATEGORY_GWW4FYB,CATEGORY_CATEGORY_LXSLG6M,CATEGORY_CATEGORY_M1ZXYVG,CATEGORY_CATEGORY_R821UZV,LAPSE
986,PID_K5MU9OI,1992,NaN,2018-08-31,2018-01-05 00:00:00,1,NaN,2018,1,0,...,0,0,0,0,0,0,1,0,0,1
986,PID_K5MU9OI,1992,NaN,2018-08-31,2018-01-05 00:00:00,3,NaN,2018,1,0,...,0,0,0,0,0,1,0,0,0,1
986,PID_K5MU9OI,1992,NaN,2018-08-31,2018-01-05 00:00:00,3,NaN,2018,1,0,...,0,0,0,0,0,1,0,0,0,1
986,PID_K5MU9OI,1992,NaN,2018-08-31,2018-01-05 00:00:00,1,45513.429325,2018,1,0,...,1,0,0,0,0,0,1,0,0,1
986,PID_K5MU9OI,1992,NaN,2018-08-31,2018-01-05 00:00:00,3,45513.429325,2018,1,0,...,1,0,0,0,0,1,0,0,0,1
986,PID_K5MU9OI,1992,NaN,2018-08-31,2018-01-05 00:00:00,3,45513.429325,2018,1,0,...,1,0,0,0,0,1,0,0,0,1
986,PID_K5MU9OI,1992,NaN,2018-08-31,2018-01-05 00:00:00,3,NaN,2018,1,0,...,0,0,0,0,0,1,0,0,0,1
986,PID_K5MU9OI,1992,NaN,2018-08-31,2018-01-05 00:00:00,3,45513.429325,2018,1,0,...,1,0,0,0,0,1,0,0,0,1
986,PID_K5MU9OI,1992,12328.624776,2018-05-31,2018-01-05 00:00:00,1,NaN,2018,1,0,...,0,0,0,0,0,0,1,0,0,1
986,PID_K5MU9OI,1992,12328.624776,2018-05-31,2018-01-05 00:00:00,3,NaN,2018,1,0,...,0,0,0,0,0,1,0,0,0,1


In [84]:
train_df.isna().sum()


POLICY_ID                               0
BIRTH YEAR                              0
AMOUNT_PAID                          3192
DATE_PAID                               0
PREMIUM_DUE_DATE                        0
PRINC_RSHIP                             0
SUM_ASSURED                        103554
LAPSE_YEAR                              0
GENDER_F                                0
GENDER_M                                0
PRODUCT_CODE_PPR_PRODCD_165U4LY         0
PRODUCT_CODE_PPR_PRODCD_APTRA9E         0
PRODUCT_CODE_PPR_PRODCD_B2KVCE7         0
PRODUCT_CODE_PPR_PRODCD_EJ7YKFV         0
PRODUCT_CODE_PPR_PRODCD_GLE2MHV         0
PRODUCT_CODE_PPR_PRODCD_H6S21FA         0
PRODUCT_CODE_PPR_PRODCD_KOFUYNN         0
PRODUCT_CODE_PPR_PRODCD_OK3TM96         0
PRODUCT_CODE_PPR_PRODCD_W0F6GK1         0
CATEGORY_CATEGORY_1750CEH               0
CATEGORY_CATEGORY_8DALFYO               0
CATEGORY_CATEGORY_GWW4FYB               0
CATEGORY_CATEGORY_LXSLG6M               0
CATEGORY_CATEGORY_M1ZXYVG         

In [85]:
# train_df[(train_df['PREMIUM_DUE_DATE'].isna()) & (train_df['AMOUNT_PAID'] != 0)]
# train_df[(train_df['PREMIUM_DUE_DATE'].isna()) & (train_df['AMOUNT_PAID'].notnull())]
# train_df[train_df['AMOUNT_PAID'] == 0].shape
# train_df.shape
train_df['PREMIUM_DUE_DATE'].unique()

array(['2018-01-10 00:00:00', '2018-01-04 00:00:00',
       '2019-01-01 00:00:00', '2018-01-12 00:00:00',
       '2018-01-11 00:00:00', '2018-01-05 00:00:00',
       '2018-01-09 00:00:00', '2018-01-08 00:00:00',
       '2018-01-06 00:00:00', '2018-01-07 00:00:00',
       '2018-01-03 00:00:00', '2018-01-02 00:00:00',
       '2019-01-02 00:00:00', '2019-01-03 00:00:00',
       '2019-01-05 00:00:00', '2019-01-04 00:00:00',
       '2017-01-09 00:00:00', '2017-01-10 00:00:00',
       '2017-01-07 00:00:00', '2017-01-11 00:00:00',
       '2017-01-12 00:00:00', '2017-01-08 00:00:00',
       '2018-01-01 00:00:00', '2019-01-06 00:00:00'], dtype=object)

In [86]:
payment_df[payment_df['AMOUNTPAID'] != 0].shape

(278530, 5)

In [87]:
payment_df.columns

Index(['Policy ID', 'AMOUNTPAID', 'DATEPAID', 'POSTDATE', 'PREMIUMDUEDATE'], dtype='object')

In [88]:
# payment_df[payment_df['Policy ID'] == 'PID_6XDDPTV'].sort_values(by='DATEPAID')

In [89]:
# pd.set_option('display.max_rows',None)
train_df[train_df['POLICY_ID'] == 'PID_REBKSKG'].sort_values(by='DATE_PAID').head(100)

,POLICY_ID,BIRTH YEAR,AMOUNT_PAID,DATE_PAID,PREMIUM_DUE_DATE,PRINC_RSHIP,SUM_ASSURED,LAPSE_YEAR,GENDER_F,GENDER_M,...,PRODUCT_CODE_PPR_PRODCD_KOFUYNN,PRODUCT_CODE_PPR_PRODCD_OK3TM96,PRODUCT_CODE_PPR_PRODCD_W0F6GK1,CATEGORY_CATEGORY_1750CEH,CATEGORY_CATEGORY_8DALFYO,CATEGORY_CATEGORY_GWW4FYB,CATEGORY_CATEGORY_LXSLG6M,CATEGORY_CATEGORY_M1ZXYVG,CATEGORY_CATEGORY_R821UZV,LAPSE
14043,PID_REBKSKG,2008,437.696085,2018-09-28,2018-01-10 00:00:00,3,27631.086529,2019,1,0,...,0,0,0,0,0,1,0,0,0,1
14043,PID_REBKSKG,1974,437.696085,2018-09-28,2018-01-10 00:00:00,3,27631.086529,2019,1,0,...,0,0,0,0,0,1,0,0,0,1
14043,PID_REBKSKG,1974,437.696085,2018-09-28,2018-01-10 00:00:00,1,27631.086529,2019,1,0,...,0,0,0,0,0,1,0,0,0,1
14043,PID_REBKSKG,1974,437.696085,2018-09-28,2018-01-10 00:00:00,3,27631.086529,2019,1,0,...,0,0,0,0,0,1,0,0,0,1
14043,PID_REBKSKG,1974,437.696085,2018-09-28,2018-01-10 00:00:00,1,27631.086529,2019,1,0,...,0,0,0,0,0,1,0,0,0,1
14043,PID_REBKSKG,2008,437.696085,2018-09-28,2018-01-10 00:00:00,1,27631.086529,2019,1,0,...,0,0,0,0,0,1,0,0,0,1
14043,PID_REBKSKG,2008,437.696085,2018-09-28,2018-01-10 00:00:00,1,27631.086529,2019,1,0,...,0,0,0,0,0,1,0,0,0,1
14043,PID_REBKSKG,2008,437.696085,2018-09-28,2018-01-10 00:00:00,3,27631.086529,2019,1,0,...,0,0,0,0,0,1,0,0,0,1
14043,PID_REBKSKG,2008,437.696085,2018-09-28,2018-01-10 00:00:00,1,27631.086529,2019,1,0,...,0,0,0,0,0,1,0,0,0,1
14043,PID_REBKSKG,2008,437.696085,2018-09-28,2018-01-10 00:00:00,3,27631.086529,2019,1,0,...,0,0,0,0,0,1,0,0,0,1


In [90]:
train_df['DATE_PAID'] = pd.to_datetime(train_df['DATE_PAID'])


<ipython-input-90-a6bfab28aa8e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['DATE_PAID'] = pd.to_datetime(train_df['DATE_PAID'])


In [91]:
train_df.dtypes

POLICY_ID                                  object
BIRTH YEAR                                  int64
AMOUNT_PAID                               float64
DATE_PAID                          datetime64[ns]
PREMIUM_DUE_DATE                           object
PRINC_RSHIP                                 int64
SUM_ASSURED                               float64
LAPSE_YEAR                                  int64
GENDER_F                                    uint8
GENDER_M                                    uint8
PRODUCT_CODE_PPR_PRODCD_165U4LY             uint8
PRODUCT_CODE_PPR_PRODCD_APTRA9E             uint8
PRODUCT_CODE_PPR_PRODCD_B2KVCE7             uint8
PRODUCT_CODE_PPR_PRODCD_EJ7YKFV             uint8
PRODUCT_CODE_PPR_PRODCD_GLE2MHV             uint8
PRODUCT_CODE_PPR_PRODCD_H6S21FA             uint8
PRODUCT_CODE_PPR_PRODCD_KOFUYNN             uint8
PRODUCT_CODE_PPR_PRODCD_OK3TM96             uint8
PRODUCT_CODE_PPR_PRODCD_W0F6GK1             uint8
CATEGORY_CATEGORY_1750CEH                   uint8


In [92]:
train_df.head()

,POLICY_ID,BIRTH YEAR,AMOUNT_PAID,DATE_PAID,PREMIUM_DUE_DATE,PRINC_RSHIP,SUM_ASSURED,LAPSE_YEAR,GENDER_F,GENDER_M,...,PRODUCT_CODE_PPR_PRODCD_KOFUYNN,PRODUCT_CODE_PPR_PRODCD_OK3TM96,PRODUCT_CODE_PPR_PRODCD_W0F6GK1,CATEGORY_CATEGORY_1750CEH,CATEGORY_CATEGORY_8DALFYO,CATEGORY_CATEGORY_GWW4FYB,CATEGORY_CATEGORY_LXSLG6M,CATEGORY_CATEGORY_M1ZXYVG,CATEGORY_CATEGORY_R821UZV,LAPSE
16,PID_MFAAYNJ,1987,3517.916006,2018-09-29,2018-01-10 00:00:00,4,27631.086529,2019,1,0,...,1,0,0,0,0,1,0,0,0,1
16,PID_MFAAYNJ,1987,3517.916006,2018-09-29,2018-01-10 00:00:00,1,123487.430359,2019,1,0,...,1,0,0,0,0,1,0,0,0,1
16,PID_MFAAYNJ,1987,3517.916006,2018-09-29,2018-01-10 00:00:00,2,123487.430359,2019,1,0,...,1,0,0,0,0,1,0,0,0,1
16,PID_MFAAYNJ,1987,3517.916006,2018-09-29,2018-01-10 00:00:00,2,NaN,2019,1,0,...,0,0,0,0,0,1,0,0,0,1
16,PID_MFAAYNJ,1987,3517.916006,2018-09-29,2018-01-10 00:00:00,3,NaN,2019,1,0,...,0,0,0,0,0,1,0,0,0,1


In [93]:
train_df[train_df['POLICY_ID'] == 'PID_REBKSKG'].sort_values(by='DATE_PAID').head(100)

,POLICY_ID,BIRTH YEAR,AMOUNT_PAID,DATE_PAID,PREMIUM_DUE_DATE,PRINC_RSHIP,SUM_ASSURED,LAPSE_YEAR,GENDER_F,GENDER_M,...,PRODUCT_CODE_PPR_PRODCD_KOFUYNN,PRODUCT_CODE_PPR_PRODCD_OK3TM96,PRODUCT_CODE_PPR_PRODCD_W0F6GK1,CATEGORY_CATEGORY_1750CEH,CATEGORY_CATEGORY_8DALFYO,CATEGORY_CATEGORY_GWW4FYB,CATEGORY_CATEGORY_LXSLG6M,CATEGORY_CATEGORY_M1ZXYVG,CATEGORY_CATEGORY_R821UZV,LAPSE
14043,PID_REBKSKG,2008,437.696085,2018-09-28,2018-01-10 00:00:00,3,27631.086529,2019,1,0,...,0,0,0,0,0,1,0,0,0,1
14043,PID_REBKSKG,1974,437.696085,2018-09-28,2018-01-10 00:00:00,3,27631.086529,2019,1,0,...,0,0,0,0,0,1,0,0,0,1
14043,PID_REBKSKG,1974,437.696085,2018-09-28,2018-01-10 00:00:00,1,27631.086529,2019,1,0,...,0,0,0,0,0,1,0,0,0,1
14043,PID_REBKSKG,1974,437.696085,2018-09-28,2018-01-10 00:00:00,3,27631.086529,2019,1,0,...,0,0,0,0,0,1,0,0,0,1
14043,PID_REBKSKG,1974,437.696085,2018-09-28,2018-01-10 00:00:00,1,27631.086529,2019,1,0,...,0,0,0,0,0,1,0,0,0,1
14043,PID_REBKSKG,2008,437.696085,2018-09-28,2018-01-10 00:00:00,1,27631.086529,2019,1,0,...,0,0,0,0,0,1,0,0,0,1
14043,PID_REBKSKG,2008,437.696085,2018-09-28,2018-01-10 00:00:00,1,27631.086529,2019,1,0,...,0,0,0,0,0,1,0,0,0,1
14043,PID_REBKSKG,2008,437.696085,2018-09-28,2018-01-10 00:00:00,3,27631.086529,2019,1,0,...,0,0,0,0,0,1,0,0,0,1
14043,PID_REBKSKG,2008,437.696085,2018-09-28,2018-01-10 00:00:00,1,27631.086529,2019,1,0,...,0,0,0,0,0,1,0,0,0,1
14043,PID_REBKSKG,2008,437.696085,2018-09-28,2018-01-10 00:00:00,3,27631.086529,2019,1,0,...,0,0,0,0,0,1,0,0,0,1


In [94]:
# payment_df
# 1. select the Policy ID
# 2. Select the dates
# 3. Order the dates in ascending
# 4. Get the difference betweeen n & n-1
# 5. Find the average
# 6. Output a dict of Policy ID : Average data

unique_ids = list(set([id for id in train_df['POLICY_ID']]))
days_dict = {}

for policy_id in unique_ids: #Iterate through the Policy IDs

    #create DF of rows sorted by DATEPaid
    df = train_df[train_df['POLICY_ID'] == policy_id].sort_values(by='DATE_PAID')
    #print(policy_id)
    

    df = df.reset_index(drop=True)
    date_diff = []
    value = []

    #Find difference between the previous row & the current
    
    for idx, row in df.iterrows():
        if idx == 0:
            continue
        else:
            date_diff_val = (df.loc[idx,'DATE_PAID'] - df.loc[idx - 1,'DATE_PAID']).days
            
            if date_diff_val != 0:
                #print('{} '.format(df.loc[idx-1,'DATE_PAID']))
                #print('{} \n'.format(df.loc[idx,'DATE_PAID']))
                #print('{}\n'.format(date_diff_val))
                date_diff.append(date_diff_val)
    if len(date_diff) == 0:
        average = 0
    else:
        average = sum(date_diff)/len(date_diff)
    days_dict[policy_id] = average               
    # print(policy_id)        

days_dict

{'PID_CLBG0IH': 0,
 'PID_3N2IY6X': 29.666666666666668,
 'PID_B0O2N75': 0,
 'PID_2E2CZL0': 0,
 'PID_R470VX2': 30.166666666666668,
 'PID_LNXEXFG': 30.666666666666668,
 'PID_Q5RH3N6': 2.0,
 'PID_SBX7CO5': 0,
 'PID_Q1J2NH4': 38.0,
 'PID_OK5RV3N': 28.0,
 'PID_HHX6GGO': 99.0,
 'PID_F9CMPY3': 30.0,
 'PID_K87G9LL': 0,
 'PID_I7X4WGH': 0,
 'PID_S8H9H3Y': 0,
 'PID_7IG7HUV': 0,
 'PID_W47DBF9': 28.0,
 'PID_1SES0HR': 0,
 'PID_LS2AFT9': 0,
 'PID_HWDRNQH': 30.25,
 'PID_Z1CA9IQ': 42.0,
 'PID_J75CDZH': 30.8,
 'PID_5WWEMCB': 28.0,
 'PID_N7OUQYF': 0,
 'PID_Z9UQVC0': 0,
 'PID_XOU7TRM': 31.0,
 'PID_QXFCWSY': 32.0,
 'PID_J012TDJ': 0,
 'PID_5EK9GK4': 0,
 'PID_DQL75JX': 0,
 'PID_O3LEPM8': 34.666666666666664,
 'PID_6CURJPB': 0,
 'PID_K7INRK1': 0,
 'PID_W887M7Z': 28.0,
 'PID_UVZILFM': 0,
 'PID_WRA5BQN': 31.0,
 'PID_UGSBAYV': 31.5,
 'PID_BR4V8S6': 31.714285714285715,
 'PID_BJG7RI3': 0,
 'PID_CQFYP8L': 0,
 'PID_NY0SWLF': 0,
 'PID_QQAQLV4': 29.0,
 'PID_Q5M194F': 114.5,
 'PID_KUXBJ10': 30.5,
 'PID_FK9W029': 29.5,
 '

In [95]:
relationship_dict = {}

for policy_id in unique_ids: #Iterate through the Policy IDs

    #create DF of rows sorted by DATEPaid
    df = train_df[train_df['POLICY_ID'] == policy_id]
    #print(policy_id)
    
    df = df.reset_index(drop=True)
    relationship_count = []

    #Find difference between the previous row & the current
    
    for idx, row in df.iterrows():
        relationship_count.append(row['PRINC_RSHIP'])
    relation_len = len(set(relationship_count))
    relationship_dict[policy_id] = relation_len
    
relationship_dict

{'PID_CLBG0IH': 3,
 'PID_3N2IY6X': 3,
 'PID_B0O2N75': 4,
 'PID_2E2CZL0': 2,
 'PID_R470VX2': 3,
 'PID_LNXEXFG': 4,
 'PID_Q5RH3N6': 4,
 'PID_SBX7CO5': 1,
 'PID_Q1J2NH4': 1,
 'PID_OK5RV3N': 1,
 'PID_HHX6GGO': 1,
 'PID_F9CMPY3': 2,
 'PID_K87G9LL': 4,
 'PID_I7X4WGH': 3,
 'PID_S8H9H3Y': 3,
 'PID_7IG7HUV': 4,
 'PID_W47DBF9': 3,
 'PID_1SES0HR': 2,
 'PID_LS2AFT9': 3,
 'PID_HWDRNQH': 3,
 'PID_Z1CA9IQ': 3,
 'PID_J75CDZH': 2,
 'PID_5WWEMCB': 3,
 'PID_N7OUQYF': 3,
 'PID_Z9UQVC0': 3,
 'PID_XOU7TRM': 3,
 'PID_QXFCWSY': 3,
 'PID_J012TDJ': 3,
 'PID_5EK9GK4': 3,
 'PID_DQL75JX': 3,
 'PID_O3LEPM8': 3,
 'PID_6CURJPB': 2,
 'PID_K7INRK1': 3,
 'PID_W887M7Z': 4,
 'PID_UVZILFM': 3,
 'PID_WRA5BQN': 1,
 'PID_UGSBAYV': 2,
 'PID_BR4V8S6': 5,
 'PID_BJG7RI3': 2,
 'PID_CQFYP8L': 3,
 'PID_NY0SWLF': 3,
 'PID_QQAQLV4': 3,
 'PID_Q5M194F': 4,
 'PID_KUXBJ10': 3,
 'PID_FK9W029': 3,
 'PID_DRSJ47L': 2,
 'PID_6X2N2JV': 3,
 'PID_C70L5PY': 3,
 'PID_NPFV2O7': 1,
 'PID_PTAQ5QT': 3,
 'PID_01AH5OW': 3,
 'PID_Z00YW7T': 3,
 'PID_E3MPQP

In [96]:
train_df[train_df['POLICY_ID'] == 'PID_MFAAYNJ']

,POLICY_ID,BIRTH YEAR,AMOUNT_PAID,DATE_PAID,PREMIUM_DUE_DATE,PRINC_RSHIP,SUM_ASSURED,LAPSE_YEAR,GENDER_F,GENDER_M,...,PRODUCT_CODE_PPR_PRODCD_KOFUYNN,PRODUCT_CODE_PPR_PRODCD_OK3TM96,PRODUCT_CODE_PPR_PRODCD_W0F6GK1,CATEGORY_CATEGORY_1750CEH,CATEGORY_CATEGORY_8DALFYO,CATEGORY_CATEGORY_GWW4FYB,CATEGORY_CATEGORY_LXSLG6M,CATEGORY_CATEGORY_M1ZXYVG,CATEGORY_CATEGORY_R821UZV,LAPSE
16,PID_MFAAYNJ,1987,3517.916006,2018-09-29,2018-01-10 00:00:00,4,27631.086529,2019,1,0,...,1,0,0,0,0,1,0,0,0,1
16,PID_MFAAYNJ,1987,3517.916006,2018-09-29,2018-01-10 00:00:00,1,123487.430359,2019,1,0,...,1,0,0,0,0,1,0,0,0,1
16,PID_MFAAYNJ,1987,3517.916006,2018-09-29,2018-01-10 00:00:00,2,123487.430359,2019,1,0,...,1,0,0,0,0,1,0,0,0,1
16,PID_MFAAYNJ,1987,3517.916006,2018-09-29,2018-01-10 00:00:00,2,NaN,2019,1,0,...,0,0,0,0,0,1,0,0,0,1
16,PID_MFAAYNJ,1987,3517.916006,2018-09-29,2018-01-10 00:00:00,3,NaN,2019,1,0,...,0,0,0,0,0,1,0,0,0,1
16,PID_MFAAYNJ,1987,3517.916006,2018-09-29,2018-01-10 00:00:00,1,NaN,2019,1,0,...,0,0,0,0,0,1,0,0,0,1
16,PID_MFAAYNJ,1987,3517.916006,2018-09-29,2018-01-10 00:00:00,4,NaN,2019,1,0,...,0,0,0,0,0,1,0,0,0,1
16,PID_MFAAYNJ,1987,3517.916006,2018-09-29,2018-01-10 00:00:00,3,45513.429325,2019,1,0,...,1,0,0,0,0,1,0,0,0,1
16,PID_MFAAYNJ,1987,3517.916006,2018-09-29,2018-01-10 00:00:00,4,27631.086529,2019,1,0,...,1,0,0,0,0,1,0,0,0,1
16,PID_MFAAYNJ,1987,3517.916006,2018-09-29,2018-01-10 00:00:00,1,123487.430359,2019,1,0,...,1,0,0,0,0,1,0,0,0,1


In [97]:
train_df['PAY_DATES'] = train_df['POLICY_ID'].map(days_dict)
#gapminder_df['pop']= gapminder_df['continent'].map(pop_dict)

<ipython-input-97-de5e721bd4b2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['PAY_DATES'] = train_df['POLICY_ID'].map(days_dict)


In [98]:
train_df[train_df['PAY_DATES'].notnull()]

,POLICY_ID,BIRTH YEAR,AMOUNT_PAID,DATE_PAID,PREMIUM_DUE_DATE,PRINC_RSHIP,SUM_ASSURED,LAPSE_YEAR,GENDER_F,GENDER_M,...,PRODUCT_CODE_PPR_PRODCD_OK3TM96,PRODUCT_CODE_PPR_PRODCD_W0F6GK1,CATEGORY_CATEGORY_1750CEH,CATEGORY_CATEGORY_8DALFYO,CATEGORY_CATEGORY_GWW4FYB,CATEGORY_CATEGORY_LXSLG6M,CATEGORY_CATEGORY_M1ZXYVG,CATEGORY_CATEGORY_R821UZV,LAPSE,PAY_DATES
16,PID_MFAAYNJ,1987,3517.916006,2018-09-29,2018-01-10 00:00:00,4,27631.086529,2019,1,0,...,0,0,0,0,1,0,0,0,1,0.0
16,PID_MFAAYNJ,1987,3517.916006,2018-09-29,2018-01-10 00:00:00,1,123487.430359,2019,1,0,...,0,0,0,0,1,0,0,0,1,0.0
16,PID_MFAAYNJ,1987,3517.916006,2018-09-29,2018-01-10 00:00:00,2,123487.430359,2019,1,0,...,0,0,0,0,1,0,0,0,1,0.0
16,PID_MFAAYNJ,1987,3517.916006,2018-09-29,2018-01-10 00:00:00,2,NaN,2019,1,0,...,0,0,0,0,1,0,0,0,1,0.0
16,PID_MFAAYNJ,1987,3517.916006,2018-09-29,2018-01-10 00:00:00,3,NaN,2019,1,0,...,0,0,0,0,1,0,0,0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51671,PID_BS40NIJ,1984,2341.902952,2018-12-24,2019-01-01 00:00:00,2,27631.086529,2019,1,0,...,0,0,0,0,1,0,0,0,1,31.0
51671,PID_BS40NIJ,1984,2341.902952,2018-12-24,2019-01-01 00:00:00,4,NaN,2019,1,0,...,0,0,0,0,1,0,0,0,1,31.0
51671,PID_BS40NIJ,1984,2341.902952,2018-12-24,2019-01-01 00:00:00,3,27631.086529,2019,1,0,...,0,0,0,0,1,0,0,0,1,31.0
51671,PID_BS40NIJ,1984,2341.902952,2018-12-24,2019-01-01 00:00:00,1,27631.086529,2019,1,0,...,0,0,0,0,1,0,0,0,1,31.0


In [99]:
train_df['RLTSHIP_CNT'] = train_df['POLICY_ID'].map(relationship_dict)

<ipython-input-99-bb2685d3b052>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['RLTSHIP_CNT'] = train_df['POLICY_ID'].map(relationship_dict)


In [100]:
train_df.shape

(255618, 28)

In [101]:
train_df.columns

Index(['POLICY_ID', 'BIRTH YEAR', 'AMOUNT_PAID', 'DATE_PAID',
       'PREMIUM_DUE_DATE', 'PRINC_RSHIP', 'SUM_ASSURED', 'LAPSE_YEAR',
       'GENDER_F', 'GENDER_M', 'PRODUCT_CODE_PPR_PRODCD_165U4LY',
       'PRODUCT_CODE_PPR_PRODCD_APTRA9E', 'PRODUCT_CODE_PPR_PRODCD_B2KVCE7',
       'PRODUCT_CODE_PPR_PRODCD_EJ7YKFV', 'PRODUCT_CODE_PPR_PRODCD_GLE2MHV',
       'PRODUCT_CODE_PPR_PRODCD_H6S21FA', 'PRODUCT_CODE_PPR_PRODCD_KOFUYNN',
       'PRODUCT_CODE_PPR_PRODCD_OK3TM96', 'PRODUCT_CODE_PPR_PRODCD_W0F6GK1',
       'CATEGORY_CATEGORY_1750CEH', 'CATEGORY_CATEGORY_8DALFYO',
       'CATEGORY_CATEGORY_GWW4FYB', 'CATEGORY_CATEGORY_LXSLG6M',
       'CATEGORY_CATEGORY_M1ZXYVG', 'CATEGORY_CATEGORY_R821UZV', 'LAPSE',
       'PAY_DATES', 'RLTSHIP_CNT'],
      dtype='object')

In [102]:
train_df.head(2)

,POLICY_ID,BIRTH YEAR,AMOUNT_PAID,DATE_PAID,PREMIUM_DUE_DATE,PRINC_RSHIP,SUM_ASSURED,LAPSE_YEAR,GENDER_F,GENDER_M,...,PRODUCT_CODE_PPR_PRODCD_W0F6GK1,CATEGORY_CATEGORY_1750CEH,CATEGORY_CATEGORY_8DALFYO,CATEGORY_CATEGORY_GWW4FYB,CATEGORY_CATEGORY_LXSLG6M,CATEGORY_CATEGORY_M1ZXYVG,CATEGORY_CATEGORY_R821UZV,LAPSE,PAY_DATES,RLTSHIP_CNT
16,PID_MFAAYNJ,1987,3517.916006,2018-09-29,2018-01-10 00:00:00,4,27631.086529,2019,1,0,...,0,0,0,1,0,0,0,1,0.0,4
16,PID_MFAAYNJ,1987,3517.916006,2018-09-29,2018-01-10 00:00:00,1,123487.430359,2019,1,0,...,0,0,0,1,0,0,0,1,0.0,4


In [103]:
train_df = train_df.drop(['DATE_PAID', 'PREMIUM_DUE_DATE', 'SUM_ASSURED',
                         'LAPSE_YEAR', 'PRODUCT_CODE_PPR_PRODCD_APTRA9E', 'PRODUCT_CODE_PPR_PRODCD_B2KVCE7',
                           'PRODUCT_CODE_PPR_PRODCD_EJ7YKFV', 'PRODUCT_CODE_PPR_PRODCD_GLE2MHV',
                           'PRODUCT_CODE_PPR_PRODCD_H6S21FA', 'PRODUCT_CODE_PPR_PRODCD_KOFUYNN',
                           'PRODUCT_CODE_PPR_PRODCD_OK3TM96', 'PRODUCT_CODE_PPR_PRODCD_W0F6GK1'], axis = 1)

In [104]:
train_df.columns

Index(['POLICY_ID', 'BIRTH YEAR', 'AMOUNT_PAID', 'PRINC_RSHIP', 'GENDER_F',
       'GENDER_M', 'PRODUCT_CODE_PPR_PRODCD_165U4LY',
       'CATEGORY_CATEGORY_1750CEH', 'CATEGORY_CATEGORY_8DALFYO',
       'CATEGORY_CATEGORY_GWW4FYB', 'CATEGORY_CATEGORY_LXSLG6M',
       'CATEGORY_CATEGORY_M1ZXYVG', 'CATEGORY_CATEGORY_R821UZV', 'LAPSE',
       'PAY_DATES', 'RLTSHIP_CNT'],
      dtype='object')

In [115]:
len(train_df['POLICY_ID'].unique())

2256

In [106]:
#train_df.sort_values(by ='POLICY_ID', inplace = True)
#train_df.head()

In [119]:
train_df2 = train_df.drop_duplicates(subset='POLICY_ID', keep="first")

In [120]:
train_df2.isna().sum()

POLICY_ID                           0
BIRTH YEAR                          0
AMOUNT_PAID                        17
PRINC_RSHIP                         0
GENDER_F                            0
GENDER_M                            0
PRODUCT_CODE_PPR_PRODCD_165U4LY     0
CATEGORY_CATEGORY_1750CEH           0
CATEGORY_CATEGORY_8DALFYO           0
CATEGORY_CATEGORY_GWW4FYB           0
CATEGORY_CATEGORY_LXSLG6M           0
CATEGORY_CATEGORY_M1ZXYVG           0
CATEGORY_CATEGORY_R821UZV           0
LAPSE                               0
PAY_DATES                           0
RLTSHIP_CNT                         0
dtype: int64

In [121]:
train_df2 = train_df2[train_df2.AMOUNT_PAID.notnull()]

In [122]:
train_df2.isna().sum()

POLICY_ID                          0
BIRTH YEAR                         0
AMOUNT_PAID                        0
PRINC_RSHIP                        0
GENDER_F                           0
GENDER_M                           0
PRODUCT_CODE_PPR_PRODCD_165U4LY    0
CATEGORY_CATEGORY_1750CEH          0
CATEGORY_CATEGORY_8DALFYO          0
CATEGORY_CATEGORY_GWW4FYB          0
CATEGORY_CATEGORY_LXSLG6M          0
CATEGORY_CATEGORY_M1ZXYVG          0
CATEGORY_CATEGORY_R821UZV          0
LAPSE                              0
PAY_DATES                          0
RLTSHIP_CNT                        0
dtype: int64

In [124]:
train_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2239 entries, 16 to 51671
Data columns (total 16 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   POLICY_ID                        2239 non-null   object 
 1   BIRTH YEAR                       2239 non-null   int64  
 2   AMOUNT_PAID                      2239 non-null   float64
 3   PRINC_RSHIP                      2239 non-null   int64  
 4   GENDER_F                         2239 non-null   uint8  
 5   GENDER_M                         2239 non-null   uint8  
 6   PRODUCT_CODE_PPR_PRODCD_165U4LY  2239 non-null   uint8  
 7   CATEGORY_CATEGORY_1750CEH        2239 non-null   uint8  
 8   CATEGORY_CATEGORY_8DALFYO        2239 non-null   uint8  
 9   CATEGORY_CATEGORY_GWW4FYB        2239 non-null   uint8  
 10  CATEGORY_CATEGORY_LXSLG6M        2239 non-null   uint8  
 11  CATEGORY_CATEGORY_M1ZXYVG        2239 non-null   uint8  
 12  CATEGORY_CATEGORY_

In [137]:
train_df2 = train_df2.drop(['POLICY_ID'], axis =1)

In [138]:
one_svm = svm.OneClassSVM(nu = 0.1, kernel = 'rbf', gamma = 0.1)

In [139]:
y = train_df2['LAPSE']
X = train_df2.drop(['LAPSE'], axis = 1)

In [140]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [141]:
one_svm.fit(X_train, y_train)

OneClassSVM(gamma=0.1, nu=0.1)

In [143]:
pred = one_svm.predict(X_train)

In [144]:
pred = np.where(pred == -1, 0, pred)

array([ 1, -1, -1, ..., -1,  1, -1], dtype=int64)

In [145]:
log_loss(pred,y_train)

In [150]:
test_pred = one_svm.predict(X_test)
test_pred = np.where(test_pred == -1, 0, test_pred)

In [151]:
log_loss(test_pred,y_test)

34.154089653140645